In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn import svm, linear_model
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import StandardScaler,PolynomialFeatures, LabelEncoder,FunctionTransformer
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import numpy as np
from scipy import stats
%matplotlib inline
titanic = pd.read_csv('train.csv')
testdf = pd.read_csv('test.csv')
import pandas as pd
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold
from sklearn.cluster import KMeans

/home/nerd/Data/Conda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
import tensorflow as tf
import tflearn

In [3]:

titanic['FamilyCount']= (titanic['SibSp'])+(titanic['Parch'])
titanic['IsFemale'] = (titanic.Sex=='female').astype(int)
titanic['IsMale'] = (titanic.Sex=='male').astype(int)
testdf['FamilyCount']= (testdf['SibSp'])+(testdf['Parch'])
testdf['IsFemale'] = (testdf.Sex=='female').astype(int)
testdf['IsMale'] = (testdf.Sex=='male').astype(int)
titanic['Cabin'] =titanic['Cabin'].fillna(value='?')
titanic['Embarked'] =titanic['Embarked'].fillna(value='S')
titanic['Age'] =titanic['Age'].fillna(value='?')
testdf['Cabin'] =testdf['Cabin'].fillna(value='?')
testdf['Embarked'] =testdf['Embarked'].fillna(value='S')
testdf['Age'] =testdf['Age'].fillna(value='?')
testdf.loc[152,'Fare']=13.6755

In [4]:
titanic['IsAlone']= (titanic['FamilyCount']==0).astype(int)
testdf['IsAlone']= (testdf['FamilyCount']==0).astype(int)

In [5]:
def titlepull(title_string):
    title_list = title_string.split()
    for possible_title in title_list:
        if possible_title[-1] == '.':
            return possible_title
def cabinpull(cabin_string):
    if cabin_string[0]=='?':
        return '?'
    else:
        return cabin_string[0]
    

In [6]:
def ticketpull(ticketstring):
    if ticketstring == 'LINE':
        return 9
    string = ticketstring.split(' ')[-1]
    return string[0]
    

In [7]:
titanic['Deck']=titanic.Cabin.apply(cabinpull)
titanic['Title']= titanic.Name.apply(titlepull)
testdf['Deck']=testdf.Cabin.apply(cabinpull)
testdf['Title']= testdf.Name.apply(titlepull)
titanic['Ticketnum']= titanic.Ticket.apply(ticketpull)
testdf['Ticketnum']=testdf.Ticket.apply(ticketpull)


In [8]:
decks_list = ['?','C','B','D','E','F','A','G','T']

In [9]:
name_mappings = {
    'Dona.':'Royalty',
    'Countess.':'Royalty',
    'Sir.':'Royalty',
    'Jonkheer.':'Royalty',
    'Don.':'Royalty',
    'Lady.':'Royalty',
    'Ms.':'Miss.',
    'Col.':'Officer',
    'Capt.':'Officer',
    'Major.':'Officer',
    'Col.':'Officer',
    'Mlle.':'Miss.',
    'Mme.':'Mrs.',
    'Mrs.':'Mrs.',
    'Miss.':'Miss',
    'Mr.':'Mr.',
    'Master.':'Master.',
    'Dr.':'Dr.',
    'Rev.':'Rev.'
    
    
}

In [10]:
titanic['Title']= titanic['Title'].map(name_mappings)
testdf['Title']= testdf['Title'].map(name_mappings)

In [11]:
validtitles = list(set(name_mappings.values()))
titanic.FamilyCount = titanic['FamilyCount'].astype("category", categories=range(11), ordered=False)


In [12]:
testdf.FamilyCount = testdf['FamilyCount'].astype("category", categories=range(11), ordered=False)
titanic.Title = titanic['Title'].astype("category", categories=validtitles, ordered=False)
testdf.Title = testdf['Title'].astype("category", categories=validtitles, ordered=False)
titanic.Deck = titanic['Deck'].astype("category", categories=decks_list, ordered=False)
testdf.Deck = testdf['Deck'].astype("category", categories=decks_list, ordered=False)
def numeric_features(df):
    return df.select_dtypes(exclude= ['object','category'])
numeric_features_tf = FunctionTransformer(numeric_features, validate=False)

In [13]:
kmg = KMeans()
kmg.fit(numeric_features(titanic.drop('Survived', axis=1)))
titanic['cluster'] =  kmg.predict(numeric_features(titanic.drop('Survived', axis=1)))
testdf['cluster'] = kmg.predict(numeric_features(testdf))

In [14]:
age_transformers = [('numfeat' ,numeric_features_tf)]

In [15]:
fu_age_impute = FeatureUnion(age_transformers)

In [16]:
knr = KNeighborsRegressor(n_jobs=-1)
lr = LinearRegression()
ss = StandardScaler()

In [17]:
age_impute_pipe_lr =  Pipeline(
    [
        ('fu',fu_age_impute),
        ('ss',ss),
        ('lr',lr)
    ]

)
age_impute_pipe_kn =  Pipeline(
    [
        ('fu_age',fu_age_impute),
        ('ss',ss),
        ('knr',knr)
    ]

)
age_params = {
    
}
age_params2 = {'knr__n_neighbors':range(4,15)
    
}

In [18]:
gs_age_kn = GridSearchCV(age_impute_pipe_kn, param_grid=age_params2, cv=5)
gs_age_lr = GridSearchCV(age_impute_pipe_lr, param_grid=age_params, cv=5)

In [19]:
titanic_ages =titanic[(titanic['Age']!='?')]
titanic_no_ages = titanic[(titanic['Age']=='?')]

test_ages =testdf[(testdf['Age']!='?')]
test_no_ages = testdf[(testdf['Age']=='?')]


In [20]:
X_train = titanic_ages.drop(['Age','Survived'],axis=1)
y_train= titanic_ages['Age']


In [21]:
X_test1 =titanic_no_ages.drop(['Age','Survived'], axis=1)
X_test2 = test_no_ages.drop('Age', axis=1)

In [22]:
gs_age_lr.fit(X_train,y_train)
print(gs_age_lr.best_score_)

0.185117252161


In [23]:
gs_age_kn.fit(X_train,y_train)
print(gs_age_kn.best_score_)

0.155501231036


In [24]:
age_impute_pipe_kn.fit(X_train,y_train)

Pipeline(steps=[('fu_age', FeatureUnion(n_jobs=1,
       transformer_list=[('numfeat', FunctionTransformer(accept_sparse=False,
          func=<function numeric_features at 0x7f00001877b8>,
          inv_kw_args=None, inverse_func=None, kw_args=None, pass_y=False,
          validate=False))],
       transfor...kowski',
          metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
          weights='uniform'))])

In [25]:
age_impute_pipe_kn.score(X_train,y_train)

0.44565589418749163

In [26]:
X_test1['Age'] = gs_age_kn.predict(X_test1)
X_test2['Age']= gs_age_kn.predict(X_test2)


In [27]:
titanic.loc[X_test1.index,'Age'] = X_test1['Age']
testdf.loc[X_test2.index,'Age'] = X_test2['Age']

In [28]:
def ageband(age):
    ageband= (int(age)//13)+1
    return ageband

In [29]:
titanic['AgeBand']=titanic.Age.apply(ageband)
testdf['AgeBand']=testdf.Age.apply(ageband)

In [30]:
def findsurname(wife):
    if wife[-1]==')':
        return wife.split()[0]
    
def findhusband(wife, husband):
    for num in range(len(wife.split())):
        if wife.split()[num][-1]=='.':
            try:
                return (wife.split()[num-1]==husband.split()[num-1])  & (wife.split()[num+1]==husband.split()[num+1])
            except:
                return False
            
        #Splitting by gender
def dead_husband(wife,husbandlist):
    #this takes a wife's name as an argument and a listas the second argument.  it returns a true or false
    result= False # assume that the wife did not die
    for num in range(len(husbandlist)):
        if findhusband(wife, husbandlist[num]):
            result = True
    return result
def dead_wife(husband,wifelist):
    result= False # assume that the wife did not die
    for num in range(len(wifelist)):
        if findhusband(wifelist[num], husband):
            result = True
    return result
            
titanicmale=  titanic[titanic['Sex']=='male'] 
titanicfemale= titanic[titanic['Sex']=='female']
testmale=  testdf[testdf['Sex']=='male'] 
testfemale= testdf[testdf['Sex']=='female']


In [31]:
livemen= titanicmale[titanicmale['Survived']==1]
deadmen= titanicmale[titanicmale['Survived']==0]
livewomen= titanicfemale[titanicfemale['Survived']==1]
deadwomen= titanicfemale[titanicfemale['Survived']==0]
deadwomennames= deadwomen.Name.values
livewomennames= livewomen.Name.values
livemennames = livemen.Name.values
deadmennames = deadmen.Name.values

In [32]:
# lets do women with dead husbands. 
titanicfemale['Dead_Husband']= titanicfemale.Name.apply(lambda x:  dead_husband(x,deadmennames))
titanicfemale['Live_Husband']= titanicfemale.Name.apply(lambda x:  dead_husband(x,livemennames))
titanicmale['Dead_Wife']=titanicmale.Name.apply(lambda x:  dead_wife(x,deadwomennames))
titanicmale['Live_Wife']=titanicmale.Name.apply(lambda x:  dead_wife(x,livewomennames))
# lets do women with dead husbands. 
testfemale['Dead_Husband']= testfemale.Name.apply(lambda x:  dead_husband(x,deadmennames))
testfemale['Live_Husband']= testfemale.Name.apply(lambda x:  dead_husband(x,livemennames))
testmale['Dead_Wife']=testmale.Name.apply(lambda x:  dead_wife(x,deadwomennames))
testmale['Live_Wife']=testmale.Name.apply(lambda x:  dead_wife(x,livewomennames))

/home/nerd/Data/Conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/nerd/Data/Conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/nerd/Data/Conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


In [33]:
testdf.loc[testfemale.index,'Dead_Husband']= testfemale['Dead_Husband'].astype(int)

testdf.loc[testfemale.index,'Live_Husband']= testfemale['Live_Husband'].astype(int)
testdf.loc[testmale.index,'Dead_Wife']= testmale['Dead_Wife'].astype(int)
testdf.loc[testmale.index,'Live_Wife']= testmale['Live_Wife'].astype(int)
titanic.loc[titanicfemale.index,'Dead_Husband']= titanicfemale['Dead_Husband'].astype(int)
titanic.loc[titanicfemale.index,'Live_Husband']= titanicfemale['Live_Husband'].astype(int)
titanic.loc[titanicmale.index,'Dead_Wife']= titanicmale['Dead_Wife'].astype(int)
titanic.loc[titanicmale.index,'Live_Wife']= titanicmale['Live_Wife'].astype(int)
titanic['Dead_Husband'].fillna(value=0, inplace=True)
titanic['Dead_Wife'].fillna(value=0, inplace=True)
testdf['Dead_Husband'].fillna(value=0, inplace=True)
testdf['Dead_Wife'].fillna(value=0, inplace=True)
titanic['Live_Husband'].fillna(value=0, inplace=True)
titanic['Live_Wife'].fillna(value=0, inplace=True)
testdf['Live_Husband'].fillna(value=0, inplace=True)
testdf['Live_Wife'].fillna(value=0, inplace=True)

In [34]:
def dummies2(df):
    cols = [ 'Embarked','Deck', 'Title', 'Ticketnum']
    return pd.get_dummies(df[cols], columns=cols)
dummies_tf2 = FunctionTransformer(dummies2, validate=False)
def numeric_features2(df):
    return df[['Pclass', 'IsFemale',
       'AgeBand',  'Ticketnum']]
def kclass (df):
    return kmc.fit_predict(numeric_features(df))
kclass_tf = FunctionTransformer(kclass, validate=False)
numeric_features_tf2 = FunctionTransformer(numeric_features2, validate=False)

In [35]:
survive_transformers = [('numfeat' ,numeric_features_tf),('dummies' ,dummies_tf2),]


In [36]:
prep_for_tensor=  FeatureUnion(survive_transformers)

In [37]:
prep_for_tensor.fit(titanic)

FeatureUnion(n_jobs=1,
       transformer_list=[('numfeat', FunctionTransformer(accept_sparse=False,
          func=<function numeric_features at 0x7f00001877b8>,
          inv_kw_args=None, inverse_func=None, kw_args=None, pass_y=False,
          validate=False)), ('dummies', FunctionTransformer(accept_sparse=False,
          func=<function dummies2 at 0x7effdc7a8400>, inv_kw_args=None,
          inverse_func=None, kw_args=None, pass_y=False, validate=False))],
       transformer_weights=None)

In [38]:
prep_for_tensor.transform(titanic)

array([[   1.,    0.,    3., ...,    0.,    0.,    0.],
       [   2.,    1.,    1., ...,    0.,    0.,    0.],
       [   3.,    1.,    3., ...,    0.,    0.,    0.],
       ..., 
       [ 889.,    0.,    3., ...,    0.,    0.,    0.],
       [ 890.,    1.,    1., ...,    0.,    0.,    0.],
       [ 891.,    0.,    3., ...,    0.,    0.,    0.]])

In [39]:
pca = PCA()

In [40]:
survive_pipe_kn =  Pipeline([('prep',prep_for_tensor),('ss',ss),('pca',pca)])

In [41]:
prep_for_tensor.fit_transform(titanic)

array([[   1.,    0.,    3., ...,    0.,    0.,    0.],
       [   2.,    1.,    1., ...,    0.,    0.,    0.],
       [   3.,    1.,    3., ...,    0.,    0.,    0.],
       ..., 
       [ 889.,    0.,    3., ...,    0.,    0.,    0.],
       [ 890.,    1.,    1., ...,    0.,    0.,    0.],
       [ 891.,    0.,    3., ...,    0.,    0.,    0.]])

In [42]:
kmc = KMeans()


In [43]:
survive_pipe_kn.fit_transform(titanic)

array([[ -1.66443754e+00,  -6.19425850e-01,   1.43959061e+00, ...,
         -4.52376226e-16,  -2.90409480e-16,  -6.20605741e-17],
       [  5.04488097e+00,   1.33510314e+00,   7.20550849e-02, ...,
          5.59327020e-16,   1.60730533e-15,   3.46704999e-16],
       [  5.81730634e-01,  -2.34880951e+00,  -2.18588900e+00, ...,
          1.21684040e-15,  -1.83253738e-16,  -4.59598755e-16],
       ..., 
       [  1.02112803e+00,  -2.63473730e+00,   5.68028058e-01, ...,
         -1.18140156e-17,   8.82934824e-17,   1.39518056e-17],
       [  1.51728641e+00,   3.95044995e+00,  -3.71304984e-01, ...,
          4.33612752e-16,   9.39921308e-17,   5.56383039e-17],
       [ -2.51072495e+00,  -4.75171768e-02,  -1.67245215e+00, ...,
          1.35486539e-16,  -1.51429344e-16,   6.54011522e-17]])

In [44]:
X=survive_pipe_kn.fit_transform(titanic.drop(['Survived','PassengerId'], axis=1))

In [45]:
X_test = survive_pipe_kn.fit_transform(testdf.drop('PassengerId', axis=1))

In [46]:
y= titanic['Survived']

In [47]:
#X = numeric_features(titanic.drop('Survived', axis=1))
kf = StratifiedKFold(random_state=2003)

In [48]:
for train,test in kf.split(X,y):
    X_train=X[train,:]
    X_test=X[test,:]
    y_train=y[train]
    y_test=y[test]
    

In [49]:
feats = X_train.shape[1]

In [50]:
# Build neural network with tflearn via tflearn tutorial

net = tflearn.input_data(shape=[None, feats])
net = tflearn.fully_connected(net, feats)
net = tflearn.dropout(net, 0.5)
net = tflearn.fully_connected(net, feats)
net = tflearn.fully_connected(net, feats)
net = tflearn.fully_connected(net, feats)
net = tflearn.fully_connected(net, 2, activation='softmax')
net = tflearn.regression(net)

In [51]:
# Define model

model = tflearn.DNN(net)
params = {}

In [52]:
tensorgs = GridSearchCV(model,param_grid=params,cv=9)

In [53]:
X.shape

(891, 44)

In [54]:
labels2 = pd.get_dummies(y_train)

In [55]:
tf.reset_default_graph()
# Start training (apply gradient descent algorithm)
model.fit(X_train,labels2.values,batch_size=200, n_epoch=1300, show_metric=True)

Training Step: 3899  | total loss: 0.46240 | time: 0.008s
| Adam | epoch: 1300 | loss: 0.46240 - acc: 0.8144 -- iter: 400/594
Training Step: 3900  | total loss: 0.52176 | time: 0.011s
| Adam | epoch: 1300 | loss: 0.52176 - acc: 0.7783 -- iter: 594/594
--


In [56]:
finalpred =model.predict(X_test)

In [57]:
finalpred

array([[ 0.87323499,  0.12676504],
       [ 0.32296097,  0.67703903],
       [ 0.1953344 ,  0.80466557],
       [ 0.05460249,  0.94539756],
       [ 0.83342522,  0.16657481],
       [ 0.15562592,  0.84437406],
       [ 0.29676452,  0.70323545],
       [ 0.95475739,  0.04524265],
       [ 0.28826299,  0.71173704],
       [ 0.91113991,  0.08886008],
       [ 0.67385703,  0.32614291],
       [ 0.91660762,  0.08339235],
       [ 0.88386583,  0.11613411],
       [ 0.66622782,  0.33377221],
       [ 0.24252246,  0.75747758],
       [ 0.20156726,  0.79843271],
       [ 0.60478592,  0.39521405],
       [ 0.88386101,  0.11613906],
       [ 0.34878385,  0.65121621],
       [ 0.83974713,  0.16025287],
       [ 0.88386679,  0.11613328],
       [ 0.38102782,  0.61897218],
       [ 0.95806354,  0.04193648],
       [ 0.34519863,  0.65480137],
       [ 0.21839528,  0.78160471],
       [ 0.83895957,  0.16104043],
       [ 0.85355866,  0.14644131],
       [ 0.57868874,  0.42131126],
       [ 0.89310551,

In [58]:
pred = finalpred[:,1].round()


In [59]:
cols = ['TN1','TN2','TN3']
shape = (train.shape[0], len(cols))
zeros= np.zeros(shape)
stacked_train = pd.DataFrame(zeros,columns = cols)

In [60]:
stacked_train.head()

,TN1,TN2,TN3
0,0.0,0.0,0.0
1,0.0,0.0,0.0
2,0.0,0.0,0.0
3,0.0,0.0,0.0
4,0.0,0.0,0.0


In [61]:
pca2 = PCA(n_components=15)
def numeric_features3(df):
    return df[['Pclass', 'IsFemale',
       'AgeBand',  'Ticketnum', 'Fare']]
numeric_features_tf3 = FunctionTransformer(numeric_features3, validate=False)
survive_transformers2 = [('numfeat' ,numeric_features_tf3),('dummies' ,dummies_tf2)]
prep_for_tensor2=  FeatureUnion(survive_transformers2)
survive_pipe2=  Pipeline([('prep',prep_for_tensor2),('ss',ss),('pca',pca2)])

In [62]:
net2 = tflearn.input_data(shape=[None, feats])
net2 = tflearn.fully_connected(net2, feats)
net2 = tflearn.dropout(net2, 0.5)
net2 = tflearn.fully_connected(net2, feats)
net2 = tflearn.fully_connected(net2, feats)
net2 = tflearn.fully_connected(net2, feats)
net2 = tflearn.fully_connected(net2, 2, activation='softmax')
net2 = tflearn.regression(net2)

In [63]:

model2 = tflearn.DNN(net2)
params = {}

In [64]:
tf.reset_default_graph()
# Start training (apply gradient descent algorithm)
model2.fit(X_train,labels2.values,batch_size=891, n_epoch=1300, show_metric=True)

---------------------------------
Run id: O3QIX3
Log directory: /tmp/tflearn_logs/
INFO:tensorflow:Summary name Accuracy/ (raw) is illegal; using Accuracy/__raw_ instead.
---------------------------------
Training samples: 594
Validation samples: 0
--
Training Step: 1  | time: 0.056s
| Adam | epoch: 001 | loss: 0.00000 - acc: 0.0000 -- iter: 594/594
--
Training Step: 2  | total loss: 0.62383 | time: 0.004s
| Adam | epoch: 002 | loss: 0.62383 - acc: 0.4455 -- iter: 594/594
--
Training Step: 3  | total loss: 0.68027 | time: 0.004s
| Adam | epoch: 003 | loss: 0.68027 - acc: 0.5851 -- iter: 594/594
--
Training Step: 4  | total loss: 0.68939 | time: 0.004s
| Adam | epoch: 004 | loss: 0.68939 - acc: 0.6084 -- iter: 594/594
--
Training Step: 5  | total loss: 0.69121 | time: 0.004s
| Adam | epoch: 005 | loss: 0.69121 - acc: 0.6138 -- iter: 594/594
--
Training Step: 6  | total loss: 0.69144 | time: 0.004s
| Adam | epoch: 006 | loss: 0.69144 - acc: 0.6153 -- iter: 594/594
--
Training Step: 7  | 

Training Step: 57  | total loss: 0.46451 | time: 0.003s
| Adam | epoch: 057 | loss: 0.46451 - acc: 0.8154 -- iter: 594/594
--
Training Step: 58  | total loss: 0.52257 | time: 0.004s
| Adam | epoch: 058 | loss: 0.52257 - acc: 0.7777 -- iter: 594/594
--
Training Step: 59  | total loss: 0.50798 | time: 0.004s
| Adam | epoch: 059 | loss: 0.50798 - acc: 0.7868 -- iter: 594/594
--
Training Step: 60  | total loss: 0.56174 | time: 0.004s
| Adam | epoch: 060 | loss: 0.56174 - acc: 0.7542 -- iter: 594/594
--
Training Step: 61  | total loss: 0.54334 | time: 0.004s
| Adam | epoch: 061 | loss: 0.54334 - acc: 0.7667 -- iter: 594/594
--
Training Step: 62  | total loss: 0.52695 | time: 0.004s
| Adam | epoch: 062 | loss: 0.52695 - acc: 0.7763 -- iter: 594/594
--
Training Step: 63  | total loss: 0.51340 | time: 0.004s
| Adam | epoch: 063 | loss: 0.51340 - acc: 0.7851 -- iter: 594/594
--
Training Step: 64  | total loss: 0.55823 | time: 0.004s
| Adam | epoch: 064 | loss: 0.55823 - acc: 0.7549 -- iter: 594

Training Step: 115  | total loss: 0.55522 | time: 0.004s
| Adam | epoch: 115 | loss: 0.55522 - acc: 0.7514 -- iter: 594/594
--
Training Step: 116  | total loss: 0.54437 | time: 0.004s
| Adam | epoch: 116 | loss: 0.54437 - acc: 0.7605 -- iter: 594/594
--
Training Step: 117  | total loss: 0.53444 | time: 0.004s
| Adam | epoch: 117 | loss: 0.53444 - acc: 0.7682 -- iter: 594/594
--
Training Step: 118  | total loss: 0.52589 | time: 0.004s
| Adam | epoch: 118 | loss: 0.52589 - acc: 0.7749 -- iter: 594/594
--
Training Step: 119  | total loss: 0.51775 | time: 0.004s
| Adam | epoch: 119 | loss: 0.51775 - acc: 0.7814 -- iter: 594/594
--
Training Step: 120  | total loss: 0.51036 | time: 0.004s
| Adam | epoch: 120 | loss: 0.51036 - acc: 0.7875 -- iter: 594/594
--
Training Step: 121  | total loss: 0.50293 | time: 0.004s
| Adam | epoch: 121 | loss: 0.50293 - acc: 0.7936 -- iter: 594/594
--
Training Step: 122  | total loss: 0.53498 | time: 0.004s
| Adam | epoch: 122 | loss: 0.53498 - acc: 0.7683 -- i

Training Step: 172  | total loss: 0.50557 | time: 0.004s
| Adam | epoch: 172 | loss: 0.50557 - acc: 0.7896 -- iter: 594/594
--
Training Step: 173  | total loss: 0.49576 | time: 0.004s
| Adam | epoch: 173 | loss: 0.49576 - acc: 0.7950 -- iter: 594/594
--
Training Step: 174  | total loss: 0.48660 | time: 0.004s
| Adam | epoch: 174 | loss: 0.48660 - acc: 0.7996 -- iter: 594/594
--
Training Step: 175  | total loss: 0.47890 | time: 0.004s
| Adam | epoch: 175 | loss: 0.47890 - acc: 0.8045 -- iter: 594/594
--
Training Step: 176  | total loss: 0.47098 | time: 0.004s
| Adam | epoch: 176 | loss: 0.47098 - acc: 0.8091 -- iter: 594/594
--
Training Step: 177  | total loss: 0.46414 | time: 0.004s
| Adam | epoch: 177 | loss: 0.46414 - acc: 0.8134 -- iter: 594/594
--
Training Step: 178  | total loss: 0.45807 | time: 0.004s
| Adam | epoch: 178 | loss: 0.45807 - acc: 0.8167 -- iter: 594/594
--
Training Step: 179  | total loss: 0.45308 | time: 0.004s
| Adam | epoch: 179 | loss: 0.45308 - acc: 0.8199 -- i

Training Step: 229  | total loss: 0.51733 | time: 0.003s
| Adam | epoch: 229 | loss: 0.51733 - acc: 0.7789 -- iter: 594/594
--
Training Step: 230  | total loss: 0.50787 | time: 0.004s
| Adam | epoch: 230 | loss: 0.50787 - acc: 0.7852 -- iter: 594/594
--
Training Step: 231  | total loss: 0.49935 | time: 0.004s
| Adam | epoch: 231 | loss: 0.49935 - acc: 0.7912 -- iter: 594/594
--
Training Step: 232  | total loss: 0.53318 | time: 0.004s
| Adam | epoch: 232 | loss: 0.53318 - acc: 0.7683 -- iter: 594/594
--
Training Step: 233  | total loss: 0.52226 | time: 0.004s
| Adam | epoch: 233 | loss: 0.52226 - acc: 0.7755 -- iter: 594/594
--
Training Step: 234  | total loss: 0.55605 | time: 0.004s
| Adam | epoch: 234 | loss: 0.55605 - acc: 0.7508 -- iter: 594/594
--
Training Step: 235  | total loss: 0.54325 | time: 0.004s
| Adam | epoch: 235 | loss: 0.54325 - acc: 0.7596 -- iter: 594/594
--
Training Step: 236  | total loss: 0.57497 | time: 0.004s
| Adam | epoch: 236 | loss: 0.57497 - acc: 0.7370 -- i

...

Training Step: 1027  | total loss: 0.56016 | time: 0.004s
| Adam | epoch: 1027 | loss: 0.56016 - acc: 0.7446 -- iter: 594/594
--
Training Step: 1028  | total loss: 0.58441 | time: 0.005s
| Adam | epoch: 1028 | loss: 0.58441 - acc: 0.7262 -- iter: 594/594
--
Training Step: 1029  | total loss: 0.56945 | time: 0.004s
| Adam | epoch: 1029 | loss: 0.56945 - acc: 0.7379 -- iter: 594/594
--
Training Step: 1030  | total loss: 0.55588 | time: 0.004s
| Adam | epoch: 1030 | loss: 0.55588 - acc: 0.7483 -- iter: 594/594
--
Training Step: 1031  | total loss: 0.54428 | time: 0.003s
| Adam | epoch: 1031 | loss: 0.54428 - acc: 0.7583 -- iter: 594/594
--
Training Step: 1032  | total loss: 0.53324 | time: 0.004s
| Adam | epoch: 1032 | loss: 0.53324 - acc: 0.7663 -- iter: 594/594
--
Training Step: 1033  | total loss: 0.52330 | time: 0.004s
| Adam | epoch: 1033 | loss: 0.52330 - acc: 0.7746 -- iter: 594/594
--
Training Step: 1034  | total loss: 0.55517 | time: 0.004s
| Adam | epoch: 1034 | loss: 0.55517 - 

Training Step: 1084  | total loss: 0.54964 | time: 0.004s
| Adam | epoch: 1084 | loss: 0.54964 - acc: 0.7516 -- iter: 594/594
--
Training Step: 1085  | total loss: 0.54001 | time: 0.004s
| Adam | epoch: 1085 | loss: 0.54001 - acc: 0.7606 -- iter: 594/594
--
Training Step: 1086  | total loss: 0.56600 | time: 0.004s
| Adam | epoch: 1086 | loss: 0.56600 - acc: 0.7371 -- iter: 594/594
--
Training Step: 1087  | total loss: 0.55468 | time: 0.004s
| Adam | epoch: 1087 | loss: 0.55468 - acc: 0.7477 -- iter: 594/594
--
Training Step: 1088  | total loss: 0.57547 | time: 0.004s
| Adam | epoch: 1088 | loss: 0.57547 - acc: 0.7263 -- iter: 594/594
--
Training Step: 1089  | total loss: 0.56328 | time: 0.004s
| Adam | epoch: 1089 | loss: 0.56328 - acc: 0.7375 -- iter: 594/594
--
Training Step: 1090  | total loss: 0.58346 | time: 0.004s
| Adam | epoch: 1090 | loss: 0.58346 - acc: 0.7168 -- iter: 594/594
--
Training Step: 1091  | total loss: 0.57089 | time: 0.005s
| Adam | epoch: 1091 | loss: 0.57089 - 

Training Step: 1141  | total loss: 0.48620 | time: 0.003s
| Adam | epoch: 1141 | loss: 0.48620 - acc: 0.8061 -- iter: 594/594
--
Training Step: 1142  | total loss: 0.51507 | time: 0.004s
| Adam | epoch: 1142 | loss: 0.51507 - acc: 0.7828 -- iter: 594/594
--
Training Step: 1143  | total loss: 0.50601 | time: 0.004s
| Adam | epoch: 1143 | loss: 0.50601 - acc: 0.7887 -- iter: 594/594
--
Training Step: 1144  | total loss: 0.49778 | time: 0.004s
| Adam | epoch: 1144 | loss: 0.49778 - acc: 0.7940 -- iter: 594/594
--
Training Step: 1145  | total loss: 0.48955 | time: 0.003s
| Adam | epoch: 1145 | loss: 0.48955 - acc: 0.7998 -- iter: 594/594
--
Training Step: 1146  | total loss: 0.48180 | time: 0.004s
| Adam | epoch: 1146 | loss: 0.48180 - acc: 0.8048 -- iter: 594/594
--
Training Step: 1147  | total loss: 0.47472 | time: 0.004s
| Adam | epoch: 1147 | loss: 0.47472 - acc: 0.8087 -- iter: 594/594
--
Training Step: 1148  | total loss: 0.46849 | time: 0.004s
| Adam | epoch: 1148 | loss: 0.46849 - 

Training Step: 1198  | total loss: 0.59396 | time: 0.004s
| Adam | epoch: 1198 | loss: 0.59396 - acc: 0.7085 -- iter: 594/594
--
Training Step: 1199  | total loss: 0.57956 | time: 0.003s
| Adam | epoch: 1199 | loss: 0.57956 - acc: 0.7222 -- iter: 594/594
--
Training Step: 1200  | total loss: 0.56639 | time: 0.004s
| Adam | epoch: 1200 | loss: 0.56639 - acc: 0.7345 -- iter: 594/594
--
Training Step: 1201  | total loss: 0.55494 | time: 0.004s
| Adam | epoch: 1201 | loss: 0.55494 - acc: 0.7450 -- iter: 594/594
--
Training Step: 1202  | total loss: 0.54418 | time: 0.004s
| Adam | epoch: 1202 | loss: 0.54418 - acc: 0.7540 -- iter: 594/594
--
Training Step: 1203  | total loss: 0.53455 | time: 0.004s
| Adam | epoch: 1203 | loss: 0.53455 - acc: 0.7626 -- iter: 594/594
--
Training Step: 1204  | total loss: 0.52541 | time: 0.004s
| Adam | epoch: 1204 | loss: 0.52541 - acc: 0.7705 -- iter: 594/594
--
Training Step: 1205  | total loss: 0.51696 | time: 0.004s
| Adam | epoch: 1205 | loss: 0.51696 - 

Training Step: 1255  | total loss: 0.54002 | time: 0.004s
| Adam | epoch: 1255 | loss: 0.54002 - acc: 0.7652 -- iter: 594/594
--
Training Step: 1256  | total loss: 0.52796 | time: 0.005s
| Adam | epoch: 1256 | loss: 0.52796 - acc: 0.7737 -- iter: 594/594
--
Training Step: 1257  | total loss: 0.51618 | time: 0.003s
| Adam | epoch: 1257 | loss: 0.51618 - acc: 0.7810 -- iter: 594/594
--
Training Step: 1258  | total loss: 0.50570 | time: 0.003s
| Adam | epoch: 1258 | loss: 0.50570 - acc: 0.7876 -- iter: 594/594
--
Training Step: 1259  | total loss: 0.49666 | time: 0.003s
| Adam | epoch: 1259 | loss: 0.49666 - acc: 0.7935 -- iter: 594/594
--
Training Step: 1260  | total loss: 0.48870 | time: 0.004s
| Adam | epoch: 1260 | loss: 0.48870 - acc: 0.7984 -- iter: 594/594
--
Training Step: 1261  | total loss: 0.48084 | time: 0.004s
| Adam | epoch: 1261 | loss: 0.48084 - acc: 0.8030 -- iter: 594/594
--
Training Step: 1262  | total loss: 0.47338 | time: 0.004s
| Adam | epoch: 1262 | loss: 0.47338 - 

In [65]:
submission  = pd.DataFrame({'Survived':pred}, index=testdf.PassengerId.values)

ValueError: Shape of passed values is (1, 297), indices imply (1, 418)

In [ ]:
submission.head()

In [ ]:
submission['Survived']=submission.Survived.apply(int)

In [ ]:
submission.head()

In [ ]:
submission.to_csv('submissiontensor12.csv', index_label='PassengerId')

In [ ]:
from tflearn.datasets import titanic as tit
tit.download_dataset('titanic_dataset.csv')

# Load CSV file, indicate that the first column represents labels
from tflearn.data_utils import load_csv
data, labels = load_csv('titanic_dataset.csv', target_column=0,
                        categorical_labels=True, n_classes=2)

In [ ]:
labels

In [ ]:
labels2 = pd.get_dummies(y)

In [ ]:
labels2.values

In [ ]:
y